In [2]:
import pandas as pd
import numpy as np
import csv
import os

In [3]:

#cambiare DATASET in base al modello che si vuole analizzare
DATASET='LSTM'

In [71]:


GT=[]
TEST=[]
VAL=[]

for i in range(1,34):  

    GT.append(np.genfromtxt(f'./SCORE_SKAB/GT/{i}.csv'))
    if(DATASET!='VAE'):
        TEST.append(np.genfromtxt(f'./SCORE_SKAB/{DATASET}/{i}.csv'))
    else:
        TEST.append(np.load(f'./SCORE_SKAB/{DATASET}/{i}.npy'))

    if(DATASET!='CONV-AE' and DATASET!='ELM'):
        VAL.append(np.load(f'./VAL_SKAB/{DATASET}/{i}.npy'))
    else:
        VAL.append(np.genfromtxt(f'./VAL_SKAB/{DATASET}/{i}.csv'))



GT=np.array(GT)
TEST=np.array(TEST)
VAL=np.array(VAL)




import csv
import pickle

header=['file','th_factor','method','value','F1','precision','recall','TP','TN','FP','FN']

with open(f'./RIS_file/{DATASET}.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    writer.writerow(header)
    thresholding_factor=[0.5,1,1.5,2]
    



    method=[0,0,0]
        
    for g in range(len(method)):
        for t in thresholding_factor:
            
            th=0
            for i in range(0,33):
                                TP=0
                                TN=0
                                FN=0
                                FP=0
                                score=VAL[i]
        
                                #IQR
                                q1, q3 = np.percentile(score, 25), np.percentile(score, 75)
                                iqr = q3 - q1
                                IQR = q3 + t* iqr

                                #MAD
                                median = np.median(score)
                                mad = 1.4826 * np.median(np.abs(score - median))
                                MAD = median + t * mad

                                #STD
                                mean, std = np.mean(score), np.std(score)
                                STD = mean + t * std

                                method=[IQR,MAD,STD]

                                #th=method[g]+th

                                out=TEST[i]>method[g]
                                le=len(TEST[i])

                                true_= GT[i]>=1
                                out=[False]*(len(true_) - len(out)) + list(out)
                                
                                prediction_ = np.array(out) 

                                TP = TP+(true_ & prediction_).sum()   
                                TN = TN+(~true_ & ~prediction_).sum()  
                                FP = FP+(~true_ & prediction_).sum()    
                                FN = FN+(true_ & ~prediction_).sum()        

                                PREC=TP / (TP + FP)
                                REC = TP/ (TP+FN)
                                f1=2 * PREC * REC/(PREC + REC)
                                m=['IQR','MAD','STD']
                                row=[i,t,m[g],method[g],f1,PREC,REC,TP,TN,FP,FN]
                                writer.writerow(row)

    #MAX

    
    tot=0
    for i in range(0,33):
                        TP=0
                        TN=0
                        FN=0
                        FP=0

                        th=np.max(VAL[i])
                        out=TEST[i]>th
                        le=len(TEST[i])

                        tot=tot+th
                        true_= GT[i]>=1

                        out=([False]*(len(true_) - len(out))) + list(out)
                        #print(out)
                        prediction_ = np.array(out) 

                        TP = TP+(true_ & prediction_).sum()   
                        TN = TN+(~true_ & ~prediction_).sum()  
                        FP = FP+(~true_ & prediction_).sum()    
                        FN = FN+(true_ & ~prediction_).sum()        

                        PREC=TP / (TP + FP)
                        REC = TP/ (TP+FN)
                        f1=2 * PREC * REC/(PREC + REC)
                        row=[i,'-','MAX',th,f1,PREC,REC,TP,TN,FP,FN]
                        writer.writerow(row)

C:\Users\gioel\AppData\Local\Temp\ipykernel_16424\578850497.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  GT=np.array(GT)
C:\Users\gioel\AppData\Local\Temp\ipykernel_16424\578850497.py:21: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  TEST=np.array(TEST)
